In [24]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Notebook_imports import *

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models


# Test the main concepts of a DL generator

A Generator yields a tuple of (x,y) and is used to feed data into a deep learning model

Each x/y tuple has the following shape: 

$batchsize * exampleSize * exampleShape$ 

--> The $exampleShape$ could be:
- for 1D vector-data: $len(vector_{data})$
- for 2D images: $height * width$: 
- for 3D volumes: $depth * height * width$)


# Interactive index and fake data generation (1D)

Please define:

- The total number of $examples$ (the size of the dataset), each $example$ represents one patient/entity
- the $exampleSize$ (number of objects per example), this could represent different timesteps of a multi input generator
- the $batchsize$, meaning how many examples should be yielded in one step

In [119]:
upper_example_limit = 100
upper_example_size_limit = 8
upper_batchsize_limit = 20
@interact
def create_fake_data(examples_=(1,upper_example_limit), example_size_=(1,upper_example_size_limit), batchsize_=(1,upper_batchsize_limit)):
    global examples, example_size, batchsize, indexes, X, Y, batches
    examples = examples_
    example_size = example_size_
    batchsize = batchsize_
    # create some fake data
    X = [list(round(np.random.rand(),2)+(delta) for i in range(example_size_)) for delta in range(examples_)]
    Y = [list(i+(delta/10) for i in range(example_size_)) for delta in range(examples_)]
    indexes = list(range(len(X)))
    batches = int(np.floor(examples/batchsize))-1

interactive(children=(IntSlider(value=50, description='examples_', min=1), IntSlider(value=4, description='exa…

In [120]:
# collect one batch
@interact
def select_batch(selected_batch = (0,batches), shuffle_indexes=False):
    global indexes
    
    if shuffle_indexes:
        random.shuffle(indexes)
        
    assert len(indexes) == examples, print('len indexes: {}, number of examples: {}'.format(len(indexes), examples))
    start_idx = selected_batch*batchsize
    end_idx = (selected_batch+1)*batchsize
    print('selected batch: {} of {} with a batchsize of {} and total {} examples'.format(selected_batch, batches, batchsize, examples))
    print('start idx: {}, end idx: {}'.format(start_idx, end_idx))
    batch_indexes = indexes[start_idx: end_idx]
    print('Indexes of the currrent batch: {}'.format(batch_indexes))
    batch_x = [X[k] for k in batch_indexes]
    batch_y = [Y[k] for k in batch_indexes]
    return([batch_x, batch_y])

interactive(children=(IntSlider(value=2, description='selected_batch', max=4), Checkbox(value=False, descripti…

# Some hardcoded examples how we construct the faxe batches

In [25]:
## we asume x and y to be a list of lists (representing the model input and output with variable length)
# create some fake data
examples = 10
example_size = 5
x = [list(i for i in range(example_size)) for _ in range(examples)]
y = [list(i+10 for i in range(example_size)) for _ in range(examples)]
print(x)
print(y)

[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4]]
[[10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14], [10, 11, 12, 13, 14]]


In [26]:
# create unique indexes, which than can be shuffeled
indexes = list(range(len(x)))
indexes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
import random
random.shuffle(indexes)
indexes

[4, 2, 1, 6, 5, 0, 8, 3, 9, 7]

In [28]:
# get the indexes for a hard-coded batchsize of two
indexes[0:2]

[4, 2]

In [29]:
# get a variable size
batchsize = 2
index = 0
batch_indexes = indexes[index*batchsize: (index+1)*batchsize]
print(batch_indexes)

[4, 2]
